In [54]:
import requests
import json
import time
from datetime import datetime
import plotly.graph_objects as go

In [55]:
#INSERT UR LOCAL TIME HERE

#datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0)
START_DATE = datetime(2024, 11, 14, 8,0)
start_time = START_DATE.timestamp()
start_time

1731589200.0

In [56]:
symbols = json.loads(requests.get('https://api.bybit.com/v5/market/tickers?category=linear').text) #turnover24h
symbols['result']
contracts = []
for e in symbols['result']['list']:
    #if e['quote_currency'] == 'USDT':
    if float(e['turnover24h']) > 50000000: #usd volume filter so u dont have so many tickers
        contracts.append(e['symbol'])
        
returns_dict = dict()

In [57]:
for e in contracts:
    
    url = f"https://api.bybit.com/v5/market/kline?symbol={e}&interval=1&limit=1&start={int(start_time)}"
    startpxresult = json.loads(requests.get(url).text)
    
    startpx = startpxresult['result']['list'][0][4]
    
    latesturl = f"https://api.bybit.com/v5/market/recent-trade?category=linear&symbol={e}&limit=1"
    endpxresult = json.loads(requests.get(latesturl).text)
    lastpx = endpxresult['result']['list'][0]['price']
    
    _return = ((float(lastpx)/float(startpx)) - 1)


    returns_dict[e] = _return

In [58]:
#returns_dict

In [59]:

data = sorted(returns_dict.items(), key = lambda k: k[1])

fig = go.Figure() 
x_data, y_data = [item[0] for item in data], [item[1] for item in data]
trace1 = go.Bar(x=x_data, y=y_data)
fig.add_trace(trace1)
fig.update_xaxes(autorange="reversed")
fig.show()